In [4]:
#!pip install control
#!pip install slycot

In [7]:
# importa os modulo necessários 

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
import control as ct


# 1 Modelagem pela módulo Sympy

In [23]:
# Definindo as variáveis simbólicas

# Constantes
M, m1, m2, m3, l1, l2,J1, J2, g, t, f0, f1, f2 = sym.symbols('M m1 m2 m3 l1 l2 J1 J2 g t  f0 f1 f2')

# Variáveis
phi1, phi2, x,u = sym.symbols(r'phi1 phi2 x u', cls=sym.Function) # funções para se trabalhar em cima


In [24]:
# Definindo as derivadas no tempo 

x = x(t)
u = u(t)
phi1 = phi1(t)
phi2 = phi2(t)

x_d = sym.diff(x, t)
phi1_d = sym.diff(phi1, t)
phi2_d = sym.diff(phi2, t)

x_dd = sym.diff(x_d, t)
phi1_dd = sym.diff(phi1_d, t)
phi2_dd = sym.diff(phi2_d, t)

dpdp2=phi2_d*phi2_d
dpdp1=phi1_d*phi1_d
dxdx=x_d*x_d

In [26]:
# Equação de Lagrange: L = T-U

L= 1/2* dxdx*(M+m1+m2)+1/2*dpdp1*(l1*l1*(m1+2*m2)+J1)+dpdp2*(m2*l2*l2+1/2*J2)+\
x_d*(-m1*l1*sym.cos(phi1)*phi1_d-2*m2*l1*sym.cos(phi1)*phi1_d-m2*l2*sym.cos(phi2)*phi2_d)+\
    phi1_d*(2*m2*l1*l2*phi2_d*sym.cos(phi1-phi2))-m1*l1*g*sym.cos(phi1)-m2*g*(2*l1*sym.cos(phi1)-l2*sym.cos(phi2))
L

-g*l1*m1*cos(phi1(t)) - g*m2*(2*l1*cos(phi1(t)) - l2*cos(phi2(t))) + 2*l1*l2*m2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + 0.5*(J1 + l1**2*(m1 + 2*m2))*Derivative(phi1(t), t)**2 + (0.5*J2 + l2**2*m2)*Derivative(phi2(t), t)**2 + 0.5*(M + m1 + m2)*Derivative(x(t), t)**2 + (-l1*m1*cos(phi1(t))*Derivative(phi1(t), t) - 2*l1*m2*cos(phi1(t))*Derivative(phi1(t), t) - l2*m2*cos(phi2(t))*Derivative(phi2(t), t))*Derivative(x(t), t)

# 2 Modelagem pelo módulo Numpy

# 3 Estabilidade